# 分离参数

In [1]:
import numpy as np
import tvm
from tvm.relax.frontend import detach_params
from tvm.script.parser import relax as R

In [2]:
@R.function
def func(x: R.Tensor((2, 3), "float32")):
    return x

In [4]:
param = tvm.nd.empty((3,), "float32")
mod = tvm.IRModule({"func": func.with_attr("params", [param])})
detached_mod, detached_params = detach_params(mod)
tvm.ir.assert_structural_equal(detached_mod, tvm.IRModule({"func": func}))
assert len(detached_params) == 1
assert "func" in detached_params
assert isinstance(detached_params["func"], list)
assert len(detached_params["func"]) == 1
np.testing.assert_allclose(detached_params["func"][0].numpy(), param.numpy())